In [19]:
import pandas as pd
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
train='https://drive.google.com/open?id=1Mskhv7GETMD_buUAJtEsKGFidJg87A2X'
test='https://drive.google.com/open?id=12fZDV8QRBlhQoLgbjiACyc5ypIcOytk5'
'''validate='https://drive.google.com/open?id=1NwHUWbbH3rxGCDBJZ7v6f86Ky9-Ve72A'''

fluff, id = train.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv')  
df1 = pd.read_csv('train.csv')

fluff, id = test.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('test.csv')  
df2 = pd.read_csv('test.csv')

'''fluff, id = validate.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('validate.csv')  
df3 = pd.read_csv('validate.csv')'''

1Mskhv7GETMD_buUAJtEsKGFidJg87A2X
12fZDV8QRBlhQoLgbjiACyc5ypIcOytk5


"fluff, id = validate.split('=')\nprint (id) # Verify that you have everything after '='\ndownloaded = drive.CreateFile({'id':id}) \ndownloaded.GetContentFile('validate.csv')  \ndf3 = pd.read_csv('validate.csv')"

In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 3000
embedding_dim = 150
#vocab_size = 5000
#embedding_dim = 150
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


sentences1 = []
labels1 = []
for sen1 in df1['Document_stp4']:
     sentences1.append((sen1))
for lab1 in df1['label']:
  if lab1=='male':
      labels1.append(0)
  elif lab1=='female':
      labels1.append(1)
sentences1=[str(s) for s in sentences1]   
#labels=[str(l) for l in labels]   
print(len(sentences1),len(labels1))

sentences2 = []
labels2 = []
for sen2 in df2['Document_stp4']:
     sentences2.append((sen2))
for lab2 in df2['label']:
  if lab2=='male':
      labels2.append(0)
  elif lab2=='female':
      labels2.append(1)
sentences2=[str(s) for s in sentences2]   
#labels=[str(l) for l in labels]   
print(len(sentences2),len(labels2))
'''sentences3 = []
labels3 = []
for sen3 in df3['Document_stp4']:
     sentences3.append((sen3))
for lab3 in df3['label']:
  if lab3=='male':
      labels3.append(0)
  elif lab3=='female':
      labels3.append(1)
sentences3=[str(s) for s in sentences3]   
#labels=[str(l) for l in labels]   
print(len(sentences3),len(labels3))'''

164800 164800
41200 41200


"sentences3 = []\nlabels3 = []\nfor sen3 in df3['Document_stp4']:\n     sentences3.append((sen3))\nfor lab3 in df3['label']:\n  if lab3=='male':\n      labels3.append(0)\n  elif lab3=='female':\n      labels3.append(1)\nsentences3=[str(s) for s in sentences3]   \n#labels=[str(l) for l in labels]   \nprint(len(sentences3),len(labels3))"

In [20]:
from numpy import array #to convert list of string to numpy array
training_sentences = array(sentences1)
testing_sentences = array(sentences2)
training_labels = array(labels1)
testing_labels = array(labels2)

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
#print(len(sentences))
#print(training_padded.shape)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
#print(testing_padded.shape)
print(training_labels)
print(len(word_index))
print(training_sentences.shape)
print(testing_labels.shape)

[0 1 0 ... 0 0 0]
202421
(164800,)
(41200,)


In [21]:
'''model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
    #(tf.keras.layers.Embedding(vocab_size, 64),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    #tf.keras.layers.Dense(64, activation='relu'),
    #tf.keras.layers.Dense(1, activation='sigmoid'))
    #tf.keras.layers.Conv1D(128, 5, activation='relu'),
    #tf.keras.layers.GlobalAveragePooling1D(),
    #tf.keras.layers.GlobalMaxPooling1D(),
    #tf.keras.layers.Dense(24, activation='relu'),
    #tf.keras.layers.Dense(1, activation='sigmoid')
])'''
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 150)          450000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                46848     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 501,073
Trainable params: 501,073
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 5
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=1)
score = model.evaluate(training_labels, testing_labels, batch_size=32)

Epoch 1/5
5150/5150 [==============================] - 465s 90ms/step - loss: 0.5817 - accuracy: 0.6654 - val_loss: 0.5619 - val_accuracy: 0.6827
Epoch 2/5
5150/5150 [==============================] - 463s 90ms/step - loss: 0.5435 - accuracy: 0.6955 - val_loss: 0.5613 - val_accuracy: 0.6826
Epoch 3/5
 252/5150 [>.............................] - ETA: 6:52 - loss: 0.5073 - accuracy: 0.7243

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, 'acc')
plot_graphs(history, 'loss')

model.save("test.h5")